### 1. 正则的几个用法

1. 匹配手机号

In [29]:
import re

phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
result = re.findall(phone_p, 'Myphone is :135-0009-4747 or 188 4786 8856 or 15878963367')
print(result)

area_pattern = re.compile('http://([\w]*)\.adjia\.cn')
result1 = re.match(area_pattern, 'http://yibin.adjia.cn/')
print(result1.group(1))



pattern_shop_count = re.compile('.*-page-([0-9]*)\.')
result = re.match(pattern_shop_count, 'list-htm-fid-1-page-26.html')
print(result.group(1))

['135-0009-4747', '188 4786 8856', '15878963367']
yibin
26


2.贪婪匹配和非贪婪匹配

- 贪婪匹配: *

- 非贪婪匹配(可能为0个)： *?

- 贪婪匹配所有字符:  .*  

- 非贪婪匹配所有字符: .*? 

In [7]:
import re
p1 = re.compile(r'我爱好.*')
result = re.match(p1, '我爱好好好好')
print(result)

p2 = re.compile(r'我爱好.*?')
result = re.match(p2, '我爱好好好好')
print(result)

<_sre.SRE_Match object; span=(0, 6), match='我爱好好好好'>
<_sre.SRE_Match object; span=(0, 3), match='我爱好'>


3.使用捕获组

括号的使用
findall可以用于找只有一个匹配内容的，相当于只有一个括号的情况，但是当有多个括号的时候，使用findall就会比较麻烦

In [8]:
p = re.compile(r'[a-z]+((\d+)-(\d+))[a-z]+')
 
result = re.match(p, 'afs123-456gds')
print(result.group()) # afs123-456gds
print(result.group(1)) # 123-456
print(result.group(2)) # 123
print(result.group(3)) # 456

afs123-456gds
123-456
123
456


4.字符串的替换

In [9]:
import re
 
p = re.compile(r'\d+')
 
result = re.sub(p, '中文', 'tom32jerry456haha')
print(result) # tom中文jerry中文haha

tom中文jerry中文haha


### 2.构建谷歌搜索函数

- 关键点1:如果实现翻墙？在本地使用了小飞机，如何让我们的python请求也经过小飞机？答案是使用代理，小飞机的代理端口是1080
所以只需要将代理指定到本地的1080即可

- 关键点2:仔细观察谷歌搜索时向服务器发送的请求，可以获得最近一年，最近一个月最近一周的结果


In [6]:
import re
import requests 
from bs4 import BeautifulSoup
proxy = {"http":"http://127.0.0.1:1080","https":"https://127.0.0.1:1080"}
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')

def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    tbs = '&tbs=qdr:y'
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
#     url = 'https://www.google.com/search?q=' + q + '&start=10&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get,proxies = proxy,verify=False)

    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.find('div', id='resultStats'))
    output = []
#     for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
#         url = searchWrapper.find('a')["href"] 
#         text = searchWrapper.find('a').text.strip()
#         result = {'text': text, 'url': url}
#         output.append(result)
    
    for content_p in soup.find_all('span', {'class':'st'}):
        text = content_p.text
        result = {'text': text}
        output.append(result)
#         print(text)
        resultm = re.findall(phone_p, text)
        print(resultm)
        print("\n")
    return output

(google("hello"))

E:\Users\binj\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
E:\Users\binj\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


None


[]

In [1]:
#coding:utf-8
import requests
import json
import time
import re

headers = {
    'Host':"map.baidu.com",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0"}

def citys():
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=s&da_src=searchBox.button&wd=%E6%B1%BD%E8%BD%A6%E7%BE%8E%E5%AE%B9%E5%BA%97&c=1&src=0&wd2=&sug=0&l=5&b=(7002451.220000001,1994587.88;19470675.22,7343963.88)&from=webmap&biz_forward={%22scaler%22:1,%22styles%22:%22pl%22}&sug_forward=&tn=B_NORMAL_MAP&nn=0&u_loc=12736591.152491,3547888.166124&ie=utf-8&t=1459951988807',headers=headers).text
    f=open('city_ids.txt','a')
    data=json.loads(html)
    for item in data['content']:
        #for city in item['city']:
            f.write(str(item)+'\n')
    f.close()

def get_infor(keyword,code,page):
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=con&from=webmap&c='+str(code)+'&wd='+keyword+'&wd2=&pn='+str(page)+'&nn='+str(page*10)+'&db=0&sug=0&addr=0&&da_src=pcmappg.poi.page&on_gel=1&src=7&gr=3&l=12&tn=B_NORMAL_MAP&u_loc=12736591.152491,3547888.166124&ie=utf-8',headers=headers).text
    data=json.loads(html)['content']
    return data


def main():
    keys=['深圳 横岗 广告']
    for keyword in keys:
        f=open(keyword+'_tels.txt','a')
        code=340
        page=1
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
                f.write(str(item['area_name'])+',    ')
                f.write(str(item['name'])+',    ')
                f.write(str(item['addr'])+',    ')
                f.write(str(item['ext']['detail_info']['phone'])+'\n')
                # print(str(item['area_name'])+'\n')

            page+=1
            print(code,page)
            time.sleep(1)
        f.close()
# main()

In [2]:
def getWeChat():
    keys=['龙岗中心城 广告']
    for keyword in keys:
        code=340
        page=1
        f=open(keyword+'_tels.txt','a')
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
#                 print(str(item['name'])+' ')
                content = '\"'+str(item['area_name'])+'\" ' + '\"' + str(item['name']) + '\"' +' \"微信\"'
                print(content)
                google(content)

            page+=1
            print(code,page)
            time.sleep(1)         
        f.close()
# getWeChat()

In [16]:
import requests
import json
import time
import re
from bs4 import BeautifulSoup
import xlwt

# get all phone number from： http://sz.adjia.cn/  ，
proxy = {"http":"http://127.0.0.1:1080","https":"https://127.0.0.1:1080"}
phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }



def get_phone(url):
    r = requests.get(url,proxies = proxy)
    soup = BeautifulSoup(r.text,"lxml")
    phone_num = soup.find('div', {'class':'telphone'})
    if phone_num.text is not None:
        resultm = re.findall(phone_p, phone_num.text)
        return resultm[0]
    else:
        return '0'


def page_h4(index_start, index_end, area_name,save_name):
    # 深圳
    url_head = 'http://{}.adjia.cn/f/'.format(area_name)
   
    excel_book = xlwt.Workbook(encoding='ascii')
    worksheet = excel_book.add_sheet('My Worksheet')
    phone_count = 0
    phone_dict={}
    for page_num in range(index_start,index_end):
        page_url = url_head + 'list-htm-fid-1-page-{}.html'.format(page_num)
#         print(page_url)
        r = requests.get(page_url, headers=headers,proxies = proxy)
        soup = BeautifulSoup(r.text,"lxml")
        for content_list in soup.find_all('div', {'class':'ListInfo'}):
            children = content_list.find_all('h4')    
            for child in children:
                print(child.text)
#                 worksheet.write(phone_count, 0, label = child.text)
                href = url_head + child.a['href']
#                 worksheet.write(phone_count, 1, label = get_phone(href))
                time.sleep(1)
                phone_dict[get_phone(href)]=child.text
               
          
    for key in phone_dict:
        worksheet.write(phone_count, 0, label = phone_dict[key])
        worksheet.write(phone_count, 1, label = key)
        phone_count+=1
        
    excel_book.save('Excel_Workbook_{}.xls'.format(save_name))
    print(len(phone_dict))
    print(phone_count)
    
        

In [17]:
def get_area_shop_count(url):
    r = requests.get(url, headers=headers,proxies = proxy)
    soup = BeautifulSoup(r.text,"lxml")
    show_page = soup.find('div', {'class':'ShowPage'})
    num = 0
    if show_page is not None:
        shop_count_index = len(show_page.find_all('a'))
        shop_count = show_page.find_all('a')[shop_count_index-2]['href']
        pattern_shop_count = re.compile('.*-page-([0-9]*)\.')
        result = re.match(pattern_shop_count, shop_count)
        num = int(result.group(1))
    
    return num
    
# get_area_shop_count('http://xuancheng.adjia.cn/f/list-htm-fid-1.html')


def get_a_area_phone(area_name):
    shop_url_head = 'http://{}.adjia.cn/f/'.format(area_name)
    shop_url = shop_url_head + 'list-htm-fid-1.html'.format(area_name)
    shop_count = get_area_shop_count(shop_url)
    page_h4(1, shop_count,area_name,area_name)
    

def get_area_name():
    start_url = 'http://www.adjia.cn/'
    area_pattern = re.compile('http://([\w]*)\.adjia\.cn')
    area_count = 0
    r = requests.get(start_url, headers=headers)
    soup = BeautifulSoup(r.text,"lxml")
    for content_list in soup.find_all('div', {'class':'area_citys'}):
        children = content_list.find_all('a')    
        for child in children:
            area_count+=1
            area_name = re.match(area_pattern, child['href'])
            area_name = (area_name.group(1))
            print(area_name)
            print(child.text)
            get_a_area_phone(area_name)
    print('total count {}'.format(area_count))

get_a_area_phone('bj')
# get_area_name()


亚克力标识标牌制作 [1图]
机器设备标牌、不锈钢牌、铜牌、钛金牌、铝牌 [4图]
上海亚克力道具制作工厂,会展有机玻璃加工 [6图]
同方全球人寿新logo更换 [4图]
高清质感画面UV平板/卷材现场打印 防水防晒 [6图]
发光字 亚克力字 门头灯箱 KT板 喷绘写真 [6图]
广告标牌 门头灯箱 招牌 发光字LED显示屏制作 [6图]
同方全球人寿新logo更换 [4图]
楼顶发光大字 楼层标识牌 公司形象墙 夜光标牌 [6图]
上海广告制作、标牌、发布、媒体推广、活动策划执行 [6图]
工厂制作广告牌发光字背景墙门头招牌LED显示屏 [6图]
楼顶大字 LOGO背景墙 LED发光字 导向牌 [6图]
广告灯箱，发光字，导视牌，指引牌，门头灯箱，形象墙 [6图]
广告灯箱，发光字，导视牌，指引牌，门头灯箱，形象墙 [6图]
楼顶发光大字 楼层标识牌 公司形象墙 夜光标牌 [6图]
上海广告制作、标牌、发布、媒体推广、活动策划执行 [6图]
工厂定制社会主义核心价值观 花草牌 垃圾桶 [6图]
上海亚克力道具制作工厂,会展有机玻璃加工 [6图]
高清质感画面UV平板/卷材现场打印 防水防晒 [6图]
立式导向牌户外广告牌、楼顶大字、标识标牌灯箱发光字 [6图]
发光字 亚克力字 门头灯箱 KT板 喷绘写真 [6图]
广告标牌 门头灯箱 招牌 发光字LED显示屏制作 [6图]
同方全球人寿新logo更换 [4图]
同方全球人寿新logo更换 [4图]
广告灯箱，发光字，导视牌，指引牌，门头灯箱，形象墙 [6图]
道具 花草牌 精神堡垒不锈钢加工定制 [6图]
楼顶发光大字 楼层标识牌 公司形象墙 夜光标牌 [6图]
双色板雕刻加工 机器标牌 电线标牌精雕双色板刻字 [2图]
上海亚克力道具制作工厂,会展有机玻璃加工 [6图]
高清质感画面UV平板/卷材现场打印 防水防晒 [6图]
立式导向牌户外广告牌、楼顶大字、标识标牌灯箱发光字 [6图]
发光字 亚克力字 门头灯箱 KT板 喷绘写真 [6图]
广告标牌 门头灯箱 招牌 发光字LED显示屏制作 [6图]
上海蕃茄农庄每人158元烧烤一日游 [3图]
开业庆典 舞台背景板搭建租赁 写真喷绘制作 易拉宝 [6图]
专业定制锌合金奖章奖牌/欧式复古奖牌制作 上海奖章 [6图]
锦旗 司标旗 国旗 奖牌 横幅 铜牌 不

亚克力制品/发光字/形象墙/喷绘招牌/户外广告 [4图]
门头 招牌 发光字 舞台 围挡 广告代工 审批发布 [2图]
闵行店面招牌制作 一一灯箱招牌一一楼顶大字 形象墙 [6图]
LED发光字门头招牌工程围挡吸塑楼梯墙体车体广告 [6图]
门头灯箱,户外广告,显示屏走字屏,亚克力,上海灯箱 [4图]
工程复印、无线胶装、企业画册、亮化工程、发光字牌 [6图]
青浦店招,广告牌,门头广告,户外广告,广告雕刻 [6图]
软膜天花 软膜喷绘 高清uv广告膜 透光膜灯箱 [6图]
上海特色墙纸.户内外广告.写真喷绘.LED发光字 [6图]
上海哪里有招牌灯箱背景墙,发光字厂家制作安装 [6图]
上海LED发光字、门头灯箱、公司形象墙logo制作 [5图]
工厂直销:室内外写真、灯箱招牌、背景墙、展架 [6图]
松江工厂制作安装广告灯箱、招牌、发光字、压克力制品 [6图]
上海门头招牌广告灯箱、LOGO背景墙、水晶字文化墙 [6图]
上海 喷绘招牌、X展架、易拉宝、背景墙、水晶字 [1图]
灯箱招牌、发光字、桁架背景及舞台搭建、展会布置 [4图]
室内外广告、楼顶大字、背景墙、发光字、展示展架 [6图]
广告字牌、灯箱、发光字、水晶字、logo形象墙 [6图]
易拉宝 灯箱 舞台背景 标牌 LED显示屏 [6图]
上海室内外广告设计、灯箱招牌、写真喷绘、发光字 [6图]
户内外灯箱广告牌 发光字 背景墙 LED显示屏 [6图]
亚克力字，公司形象墙，发光字，门头灯箱、户外广告牌 [6图]
公司logo墙 亚克力字水晶字 铭牌铜牌 玻璃贴膜 [6图]
软膜天花 软膜喷绘 高清uv广告膜 透光膜灯箱 [6图]
上海特色墙纸.户内外广告.写真喷绘.LED发光字 [6图]
工厂直销:室内外写真、灯箱招牌、背景墙、展架 [6图]
各种会务背景、舞台背景、背景板、写真、喷绘、展板、 [6图]
专业制作工程复印、无线胶装、图文设计、企业画册 [6图]
上海门头招牌广告灯箱、LOGO背景墙、水晶字文化墙 [6图]
易拉宝 灯箱 舞台背景 标牌 LED显示屏 [6图]
上海 喷绘招牌、X展架、易拉宝、背景墙、水晶字 [1图]
户外大型喷绘、户内写真喷绘、平板喷绘、车身广告喷绘 [6图]
户内外灯箱广告牌 发光字 背景墙 LED显示屏 [6图]
亚克力制品/发光字/形象墙/喷绘招牌/户外广告 

上海专业户外广告 大型楼体广告 亮化工程 优惠中 [6图]
闵行区广告标识设计制作公司,自有工厂直接加工 [3图]
发光字、灯箱招牌、LOGO墙制作安装~免费上门测量 [6图]
保温杯刻字 U盘刻字 充电宝刻字 金属刻字塑料刻字 [6图]
浦东激光打标,浦东激光雕刻,浦东激光镭雕 [5图]
无框灯箱迷你字灯箱招牌指示牌 [6图]
长宁区附近专业制作立式导向牌、制作宣传栏 [6图]
上海标牌制作、门头广告、背景墙、发光字LED灯 [6图]
楼顶发光大字 楼层标识牌 公司形象墙 夜光标牌 [6图]
厂家定制-写字楼-酒店宾馆标识牌-号码牌-科室牌 [3图]
广告灯箱，发光字，导视牌，指引牌，门头灯箱，形象墙 [6图]
长宁虹桥路 铜牌不锈钢牌 钛金牌 亚克力字 发光字 [6图]
立式导向牌户外广告牌、楼顶大字、标识标牌灯箱发光字 [6图]
双色板雕刻加工 机器标牌 电线标牌精雕双色板刻字 [2图]
长宁区附近专业制作立式导向牌、制作宣传栏 [6图]
不锈钢腐蚀铭牌 双色板精细雕刻 现场平板UV [6图]
上海标牌制作、喷绘写真、背景墙、发光字LED灯 [6图]
公司LOGO背景墙字、发光字玻璃贴膜、同行雕刻加工 [6图]
发光立体字,发光字,形象墙,宣传栏,写真喷绘雕刻等 [6图]
上海特色墙纸.户内外广告.写真喷绘.LED发光字 [6图]
青浦店招,广告牌,门头广告,户外广告,广告雕刻 [6图]
门头灯箱、形象墙、户外广告、发光字、楼顶字、宣传栏 [6图]
连锁店广告牌安装,工厂加工楼顶、户外广告、发光字等 [4图]
楼宇广告制作一展览展架一舞台搭建一喷绘写真 [6图]
办公室贴膜-汽车贴膜-雪弗板制作-批量喷绘写真 [3图]
门头招牌一发光字一灯箱一形象墙一喷绘写真一亚克力 [6图]
形象墙、广告牌、发光字、科室标牌制作公司 [6图]
室内外广告、楼顶大字、背景墙、发光字、展示展架 [6图]
写真喷绘、KT板展架、桁架搭建、车贴展览展示 [6图]
上海哪里有招牌灯箱背景墙,发光字厂家制作安装 [6图]
上海专业LED柔性霓虹灯霓虹灯LED显示屏安装制作 [6图]
上海户外写真一楼顶大字一户外广告一亮化工程 [6图]
上海专业喷绘写真LED发光字一大型户外广告招牌制作 [6图]
虹口区灯箱招牌 背景形象墙 户外广告 LED显示屏 [6图]
户内外广告牌/发光字/门头灯

上海平板UV打印|亚克力|铝合金|玻璃|大理石 [6图]
徐汇区桂林路灯箱招牌安装、广告牌安装、发光字安装 [4图]
塑料板加工雕刻，有机玻璃雕刻 [3图]
PVC会员卡、人像卡、名片、优惠卡,立马可取可快递 [6图]
无框灯箱迷你字灯箱招牌指示牌 [6图]
上海南站附近专业制作立式导向牌、制作宣传栏 [6图]
上海奉贤批发丶安装丶维修、全彩单双色 LED显示屏 [6图]
上海标牌制作、门头广告、背景墙、发光字LED灯 [6图]
铭牌，标牌激光打标刻字 [6图]
广告牌制作安装、楼顶大字、发光字、三面翻、显示屏 [6图]
上海夜光、反光标牌；腐蚀标牌；高光拉丝标牌制作 [6图]
门头发光字 标牌制作 户外广告 灯箱 显示屏 [6图]
门头发光字、LOGO背景墙、LED显示屏、标识标牌 [6图]
上海金属铭牌、PVC标牌、门牌、滴胶标牌、塑料标牌 [6图]
店面招牌,发光字,灯箱,LED,LOGO,广告牌 [6图]
长宁虹桥路 铜牌不锈钢牌 钛金牌 亚克力字 发光字 [6图]
灯箱招牌、发光字、桁架背景及舞台搭建、展会布置 [4图]
上海广告招牌 楼顶字 展板 形象墙 标识标牌 [2图]
专业UV 喷绘写真、个性背景墙DIY张贴画设计制作 [5图]
专业LED发光字,户内外写真喷绘,灯箱招牌 [4图]
上海灯箱招牌、楼顶大字、户外广告、发光字、亮化工程 [6图]
形象墙、广告牌、发光字、科室标牌制作公司 [6图]
闵行店面招牌制作 一一灯箱招牌一一楼顶大字 形象墙 [6图]
工程复印、无线胶装、企业画册、亮化工程、发光字牌 [6图]
上海灯箱、门头招牌、户外广告、发光字、LED显示屏 [6图]
户外广告牌,门头招牌,走字显示屏,发光字,背景墙 [6图]
上海门头灯箱、形象墙、户外广告、发光字、楼顶字 [6图]
LED显示屏、门头招牌、广告制作、发光字、树脂字、 [6图]
浦东金桥制作灯箱招牌 楼宇亮化灯 质优价廉让您放心 [5图]
公司LOGO背景墙字、发光字玻璃贴膜、同行雕刻加工 [6图]
海报KT板,易拉宝,X展架,海报架,拉网现场制 [5图]
嘉定周边广告牌 门头灯箱 发光字 背景墙 制作安装 [6图]
青浦店招,广告牌,门头广告,户外广告,广告雕刻 [6图]
连锁店广告牌安装,工厂加工楼顶、户外广告、发光字等 [4图]
各种会务背景、舞台背景、背景板、写真、喷

In [7]:
z = {
    1: 'a',
    2: 'a',
    3: 'a',
    4: 'b',
    5: 'c',
    6: 'c',
    7: 'd'
}


print(set(z.values()))

{'a', 'b', 'c', 'd'}
